In [ ]:
pip install tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {"peace": 0, "thumbs up": 1, "stop": 2}
filename = "keypoints_data/"
label_text = np.load(filename+"labels.npy")
label = []
for action in label_text:
    label.append(label_map[action])

In [ ]:
X = np.load(filename+"keypoints_data.npy")
y = np.array(label)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [ ]:
print(X_train.shape)
print(y_train.shape)


(679, 42)
(679,)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         (None, 42)                0         
                                                                 
 dense_3 (Dense)             (None, 20)                860       
                                                                 
 dropout_3 (Dropout)         (None, 20)                0         
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,103
Trainable params: 1,103
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)



In [ ]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
1/6 [====>.........................] - ETA: 2s - loss: 1.1757 - accuracy: 0.2734
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
6/6 [==============================] - 1s 37ms/step - loss: 1.2063 - accuracy: 0.2784 - val_loss: 1.1820 - val_accuracy: 0.3947
Epoch 2/1000
1/6 [====>.........................] - ETA: 0s - loss: 1.1833 - accuracy: 0.2578
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
6/6 [==============================] - 0s 11ms/step - loss: 1.1635 - accuracy: 0.3108 - val_loss: 1.1039 - val_accuracy: 0.3947
Epoch 3/1000
1/6 [====>.........................] - ETA: 0s - loss: 1.1532 - accuracy: 0.2969
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
6/6 [==============================] - 0s 11ms/step - loss: 1.1063 - accuracy: 0.3623 - val_loss: 1.0659 - val_accuracy: 0.3947
Epoch 4/1000
1/6 [====>.........................] - ETA: 0s - loss: 1.0947 - accuracy: 0.3906
Epoch 4: sav

In [ ]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

1/1 [==============================] - 0s 33ms/step - loss: 0.0885 - accuracy: 0.9868


In [ ]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [ ]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 87ms/step
[0.02533732 0.01266697 0.96199566]
2


In [ ]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)